In [ ]:
# execute cpm2cube.x while error message
# check output if sum > 1? -> give warning

In [1]:
import subprocess
import os
import shutil

In [2]:
def run_cpmp2cube(cpmd2cube_exe, work_dir):
    os.chdir(work_dir)
    cpmd2cube_exe = '/home/misa/opt/cpmd2cube/cpmd2cube.x'
    process = subprocess.run([cpmd2cube_exe, 'DENSITY'], capture_output = True,text=True)
    return(process)

def check_conversion(stdout, stderr):
    if stderr == 'STOP read_density_file, c0 incorrectly sized\n':
        return('c0 error')
    elif len(stdout) > 2 and len(stderr)==0:
        lines = stdout.split('\n')
        sum_squared = float(lines[2].split()[-1][:-1])
        if sum_squared < 1:
            return('reasonable density')
        else:
            return('sum^2 to large')
    else:
        return('unexpected event occured')

In [11]:
density_dirs = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/density_dirs'#'/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/density_dirs'
workdirs = []
with open(density_dirs, 'r') as f:
    for line in f:
        workdirs.append(line.strip('\n'))

In [15]:
#workdirs = ['/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/ethene_HCH/ang_2/ve_02','/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/ethene_HCH/ang_2/ve_05']
cpmd2cube_exe = '/home/misa/opt/cpmd2cube/cpmd2cube.x'
run_info = dict()
for wd in workdirs[:5]:
    # make cube file
    i = 0
    success = 'c0 error'
    while success == 'c0 error':
        print(f'Try {i}')
        process = run_cpmp2cube(cpmd2cube_exe, wd)
        success = check_conversion(process.stdout, process.stderr)
        run_info[wd] = success
        i += 1
    # move cube file
    if success == 'reasonable density':
        src = os.path.join(wd, 'DENSITY.cube') # cube path source

        # make cube dir if not existing
        cube_dir = os.path.join('/'.join(wd.split('/')[:-1]), 'cube-files')
        os.makedirs(cube_dir, exist_ok=True)

        lam_val = wd.split('/')[-1]
        cube_name = f'{lam_val}.cube' # new cube name
        dest = os.path.join(cube_dir, cube_name) # cube path destination
        
        shutil.move(src, dest)
        
        # delete DENSITY.pdb file
        density_pdb = os.path.join(wd, 'DENSITY.pdb')
        os.remove(density_pdb)
        

Try 0
Try 0
Try 0
Try 0
Try 1
Try 0
Try 1


In [13]:
for k in run_info.keys():
    if not run_info[k] == 'reasonable density':
        print(k)

In [16]:
run_info

{'/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/dihe_10.0/ve_03': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/dihe_10.0/ve_06': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/dihe_10.0/ve_08': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/dihe_10.0/ve_11': 'reasonable density',
 '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/ethane/dihe_10.0/ve_14': 'reasonable density'}